# Aggregation

### Contents

1. Aggreate functions in pandas
2. The groupby method
3. Applying functions and transformations

In [2]:
import pandas as pd

#import numpy as np

In [4]:
# reading the csv file
df=pd.read_csv('large_countries_2015.csv')
df.population = round(df.population/1000000, 1)
df

,name,population,fertility,continent
0,Bangladesh,161.0,2.12,Asia
1,Brazil,207.8,1.78,South America
2,China,1376.0,1.57,Asia
3,India,1311.1,2.43,Asia
4,Indonesia,257.6,2.28,Asia
5,Japan,126.6,1.45,Asia
6,Mexico,127.0,2.13,North America
7,Nigeria,182.2,5.89,Africa
8,Pakistan,188.9,3.04,Asia
9,Philippines,100.7,2.98,Asia


# Aggregate functions in pandas

**Aggregate function**: takes multiple rows as input and returns a single value

In [5]:
# sum()
df['population'].sum()

4504.2

In [6]:
# count()
df.count()

name          12
population    12
fertility     12
continent     12
dtype: int64

In [7]:
# aggregate functions will automatically chose the columns for which an aggreagation can be perforem
#mean()
df.mean()

/tmp/ipykernel_4054/3941788699.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.mean()


population    375.3500
fertility       2.4375
dtype: float64

In [8]:
# min()
df.min()
# careful with this, it takes them from different rows. 
# You'd need to do your selection more carefully if you 
# needed 

name          Bangladesh
population         100.7
fertility           1.45
continent         Africa
dtype: object

In [9]:
# max()
df.max()

name          United States
population           1376.0
fertility              5.89
continent     South America
dtype: object

In [22]:
# find the country with the smallest population
df.loc[df['population']==df['population'].min(),:]

,name,population,fertility,continent
9,Philippines,100.7,2.98,Asia


In [23]:
# you can make subselections of columns 
# (because we are looking at a different DataFrame now).. the one with
# repeated index and df*2
df[['population', 'fertility']].max()

population    1376.00
fertility        5.89
dtype: float64

In [24]:
# apply a predefined set of aggregate functions
df.describe()

,population,fertility
count,12.000000,12.000000
mean,375.350000,2.437500
std,456.517642,1.200781
min,100.700000,1.450000
25%,139.375000,1.737500
50%,185.550000,2.125000
75%,273.650000,2.567500
max,1376.000000,5.890000


In [25]:
# apply a customized set of aggregate functions
df[['population', 'fertility']].agg(['sum','count','mean'])

,population,fertility
sum,4504.20,29.2500
count,12.00,12.0000
mean,375.35,2.4375


In [26]:
# use a different function for each column
def double(x):
    return x * 2

df.agg({'population': 'mean', 'fertility': 'double'})

population                                               375.35
fertility     [2.12, 1.78, 1.57, 2.43, 2.28, 1.45, 2.13, 5.8...
dtype: object

# Group By in pandas

The GroupBy function works in 3 steps:

1. Split the data into groups based on some criterion
2. Apply N aggregate functions to each group independently
3. Combine the results to a single DataFrame

## Splitting

In [27]:
# Split the data set and create a new group
g1=df.groupby('continent')
g1

In [29]:
# the GroupBy object is an iterable of DataFrames
for continent, sub_df in g1:
    print(continent)
    print(sub_df)
    print('\n')

Africa
      name  population  fertility continent
7  Nigeria       182.2       5.89    Africa


Asia
          name  population  fertility continent
0   Bangladesh       161.0       2.12      Asia
2        China      1376.0       1.57      Asia
3        India      1311.1       2.43      Asia
4    Indonesia       257.6       2.28      Asia
5        Japan       126.6       1.45      Asia
8     Pakistan       188.9       3.04      Asia
9  Philippines       100.7       2.98      Asia


Europe
      name  population  fertility continent
10  Russia       143.5       1.61    Europe


North America
             name  population  fertility      continent
6          Mexico       127.0       2.13  North America
11  United States       321.8       1.97  North America


South America
     name  population  fertility      continent
1  Brazil       207.8       1.78  South America




In [30]:
# get the DataFrame of a specific group
g1.get_group('North America')

,name,population,fertility,continent
6,Mexico,127.0,2.13,North America
11,United States,321.8,1.97,North America


In [32]:
# uses the first row of each group
df.groupby('continent').first()

,name,population,fertility
continent,,,
Africa,Nigeria,182.2,5.89
Asia,Bangladesh,161.0,2.12
Europe,Russia,143.5,1.61
North America,Mexico,127.0,2.13
South America,Brazil,207.8,1.78


In [33]:
# split by an array of equal length
my_groups = list("AAAABBBBCCCC")
g2 = df.groupby(my_groups)
g2['population'].sum()

A    3055.9
B     693.4
C     754.9
Name: population, dtype: float64

In [34]:
print(g2.get_group('A'))

         name  population  fertility      continent
0  Bangladesh       161.0       2.12           Asia
1      Brazil       207.8       1.78  South America
2       China      1376.0       1.57           Asia
3       India      1311.1       2.43           Asia


In [35]:
df2 = df.set_index('name')
df2

,population,fertility,continent
name,,,
Bangladesh,161.0,2.12,Asia
Brazil,207.8,1.78,South America
China,1376.0,1.57,Asia
India,1311.1,2.43,Asia
Indonesia,257.6,2.28,Asia
Japan,126.6,1.45,Asia
Mexico,127.0,2.13,North America
Nigeria,182.2,5.89,Africa
Pakistan,188.9,3.04,Asia


In [36]:
# split by a Dictionary with keys on the Index
language = {'Bangladesh':'HD', 'Brazil':'PT', 'China':'CN',
            'India':'HD', 'Indonesia':'ID', 'Japan':'JP',
            'Mexico':'ES', 'Nigeria':'NG', 'Pakistan':'AR',
            'Philippines':'PP', 'Russia':'RU', 'United States':'EN'}
g3 = df2.groupby(language)
g3.mean()

,population,fertility
name,,
AR,188.90,3.040
CN,1376.00,1.570
EN,321.80,1.970
ES,127.00,2.130
HD,736.05,2.275
ID,257.60,2.280
JP,126.60,1.450
NG,182.20,5.890
PP,100.70,2.980


In [37]:
# split by a function - here: number of characters in the country name
g4=df2.groupby(len)
g4.mean()

,population,fertility
name,,
5,937.900000,1.816667
6,159.433333,1.840000
7,182.200000,5.890000
8,188.900000,3.040000
9,257.600000,2.280000
10,161.000000,2.120000
11,100.700000,2.980000
13,321.800000,1.970000


In [39]:
# split by a list of the above
g5=df2.groupby(['continent', language, len])
df3=g5.mean()

population  fertility
continent     name name                       
Africa        NG   7          182.2       5.89
Asia          AR   8          188.9       3.04
              CN   5         1376.0       1.57
              HD   5         1311.1       2.43
                   10         161.0       2.12
              ID   9          257.6       2.28
              JP   5          126.6       1.45
              PP   11         100.7       2.98
Europe        RU   6          143.5       1.61
North America EN   13         321.8       1.97
              ES   6          127.0       2.13
South America PT   6          207.8       1.78

In [44]:
# split by first letter
df.groupby(df['name'].str[0])['name'].count()

name
B    2
C    1
I    2
J    1
M    1
N    1
P    2
R    1
U    1
Name: name, dtype: int64

## Applying functions and transformations

### Applying Functions

apply does aggregates on a pd.Series (column)

In [45]:
# length of country names
df['name'].apply(len)

0     10
1      6
2      5
3      5
4      9
5      5
6      6
7      7
8      8
9     11
10     6
11    13
Name: name, dtype: int64

In [46]:
# using your own function on a column
def triplet(x):
    return x[:3].upper()

df['name'].apply(triplet)

0     BAN
1     BRA
2     CHI
3     IND
4     IND
5     JAP
6     MEX
7     NIG
8     PAK
9     PHI
10    RUS
11    UNI
Name: name, dtype: object

In [47]:
# using your own function on groups
def diff_from_mean(gdf):
    return gdf['population'] - gdf['population'].mean()

df.groupby('continent').apply(diff_from_mean)

continent        
Africa         7       0.000000
Asia           0    -342.128571
               2     872.871429
               3     807.971429
               4    -245.528571
               5    -376.528571
               8    -314.228571
               9    -402.428571
Europe         10      0.000000
North America  6     -97.400000
               11     97.400000
South America  1       0.000000
Name: population, dtype: float64

### Transform

Transform takes each pd.Series of a pd.DataFrame as input, 
applies a specified function to each element of the pd.Series
and returns a pd.Series of equal size.

In [48]:
# Transform
df.groupby('continent').transform('mean')

/tmp/ipykernel_4054/2755511123.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  df.groupby('continent').transform('mean')


,population,fertility
0,503.128571,2.267143
1,207.800000,1.780000
2,503.128571,2.267143
3,503.128571,2.267143
4,503.128571,2.267143
5,503.128571,2.267143
6,224.400000,2.050000
7,182.200000,5.890000
8,503.128571,2.267143
9,503.128571,2.267143


In [49]:
# Transformation by function reference
df.groupby('continent')['name'].transform(len)

0     7
1     1
2     7
3     7
4     7
5     7
6     2
7     1
8     7
9     7
10    1
11    2
Name: name, dtype: int64

In [53]:
# Transformation with your own function
def normalize(array):
    """normalize to mean 0.0 and standard deviation 1.0"""
    return (array - array.mean()) / array.std()

a=df[['population', 'fertility']].transform(normalize)
b=df[['population', 'fertility']].apply(normalize)
type(b)
type(a)


pandas.core.frame.DataFrame

In [54]:
df

,name,population,fertility,continent
0,Bangladesh,161.0,2.12,Asia
1,Brazil,207.8,1.78,South America
2,China,1376.0,1.57,Asia
3,India,1311.1,2.43,Asia
4,Indonesia,257.6,2.28,Asia
5,Japan,126.6,1.45,Asia
6,Mexico,127.0,2.13,North America
7,Nigeria,182.2,5.89,Africa
8,Pakistan,188.9,3.04,Asia
9,Philippines,100.7,2.98,Asia


In [56]:
#a=df[['population', 'fertility']].transform('mean')
b=df[['population', 'fertility']].apply('mean')
type(b)
#type(a)

pandas.core.series.Series